In [1]:
import getpass
import requests as re
import json
from datetime import datetime
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
staging_server = "https://staging-api.globalforestwatch.org"
prod_server = "https://api.resourcewatch.org"

In [3]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
def auth(env='prod'):
    serverUrl = {
        'prod': prod_server,
        'staging': staging_server
    }
    print(f'You are login into {bcolors.HEADER}{bcolors.BOLD}{env}{bcolors.ENDC}')
    with re.Session() as s:
        headers = {'Content-Type': 'application/json'}
        payload = json.dumps({ 'email': f'{input(f"Email: ")}',
                               'password': f'{getpass.getpass(prompt="Password: ")}'})
        response = s.post(f'{serverUrl[env]}/auth/login',  headers = headers,  data = payload)
        response.raise_for_status()
        print(f'{bcolors.OKGREEN}Successfully logged into {env}{bcolors.ENDC}')
    return response.json().get('data').get('token')

In [5]:
token = {
    'staging': auth('staging'),
    'prod':auth('prod')
}

You are login into staging


Email:  alicia.arenzana@vizzuality.com
Password:  ··········


Successfully logged into staging
You are login into prod


Email:  alicia.arenzana@vizzuality.com
Password:  ··········


Successfully logged into prod


In [13]:
def setTokenHeader(env, token=token):
    '''
    set up the token
    '''
    return {'Authorization':f'Bearer {token[env]}', 
            'Content-Type': 'application/json'}

def logResponseErrors(status_code, response = None, url = None, body = None):
    '''
    log errors in http calls
    '''
    if status_code !=200:
        logging.error('response: ')
        logging.error(response)
        logging.error(response.text) if response else None
        logging.error(response.json()) if response else None
        logging.error('url: ')
        logging.error(url) if url else None
        logging.error('body: ')
        logging.error(json.dumps(body)) if body else None
        
def postAssets(url, body, headers, payloads = None):
    '''
    create asset operation
    '''    
    response = re.post(url, params = payloads, data=json.dumps(body), headers = headers)
    
    logResponseErrors(response.status_code, response, url, body)
    
    response.raise_for_status()
    
    return response.json()

def createWidget(datasetId, body, toEnv = 'prod', destinationWidgetId = None):
    '''
    Copy the widget from one env to the other
    '''
    
    serverUrl = {
        'prod': prod_server,
        'staging': staging_server
    }
    
    headers = setTokenHeader(toEnv)
    url = f'{serverUrl[toEnv]}/v1/dataset/{datasetId}/widget'
    
    if destinationWidgetId:
        url = f'{url}/{destinationWidgetId}'
    
    return postAssets(url, body, headers)

In [14]:
widgetBody = json.loads('''
    {
    "name": "Reduction in Life Expectancy Map",
    "description": "",
    "source": "",
    "authors": "",
    "application": ["rw"],
    "verified": false,
    "default": false,
    "protected": false,
    "defaultEditableWidget": false,
    "published": true,
    "freeze": false,
    "env": "production",
    "queryUrl": "",
    "widgetConfig": {
        "lng": 4.504394531250001,
        "lat": -28.497660832963472,
        "zoom": 4,
        "type": "map",
        "paramsConfig": {
            "embed": {
                "src": ""
                },
            "band": null,
            "areaIntersection": null,
            "filters": [],
            "chartType": null,
            "aggregateFunction": null,
            "orderBy": null,
            "size": null,
            "color": null,
            "category": null,
            "value": null,
            "limit": 500,
            "visualizationType": "map",
            "layer_id": "b0a8f63f-e3a3-4599-abd8-19120edd98b1",
            "layer": "b0a8f63f-e3a3-4599-abd8-19120edd98b1",
            "layers": [
                "2a694289-fec9-4bfe-a6d2-56c3864ec349",
                "155968b5-7c59-4065-9e3a-0a81d52d50de",
                "00e688a8-086e-4206-9ea7-b47afca6913c",
                "7890b4f2-502a-408b-8614-3f00e0b53ec2"
                  ]
                }
            }
  }
''')

## Staging Layers OW

# Catchment Areas and Land cover

## Sediment Pressure

In [15]:
DatasetId = 'ae3ffd82-19a8-4d7c-a974-cc6d7b512f86'
layers = ['c4c7b8e1-1d74-40fe-840c-e8aa185a4461', '0f27b5ee-21e0-48d4-81ba-f8d33d773514', '9b9fe210-f99d-452f-b5d8-9bae0b1a5ec2']
widgetBody['name'] = "Sediment Pressure"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': 'ac8fb011-b003-466e-ab5e-1ccd417bbc1d',
  'type': 'widget',
  'attributes': {'name': 'Sediment Pressure',
   'dataset': 'ae3ffd82-19a8-4d7c-a974-cc6d7b512f86',
   'slug': 'Sediment-Pressure',
   'userId': '57a0aa1071e394dd32ffe137',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': 'c4c7b8e1-1d74-40fe-840c-e8a

## Water Quality Pressure

In [16]:
DatasetId = 'f7bbc39e-81ab-4d91-be37-c12e33cd8436'
layers = ['8339b4a5-b99f-4d64-8737-30f79325e106','c4c7b8e1-1d74-40fe-840c-e8aa185a4461','abefb374-3355-4110-9c03-c7031f0b61b4','fe6315f7-e208-441d-9193-9dee6499b349']
widgetBody['name'] = "Water Quality Pressure"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(DatasetId, widgetBody, 'staging')

{'data': {'id': '913ab9b7-5e5c-4830-a4c8-5e3b2d45db77',
  'type': 'widget',
  'attributes': {'name': 'Water Quality Pressure',
   'dataset': 'f7bbc39e-81ab-4d91-be37-c12e33cd8436',
   'slug': 'Water-Quality-Pressure',
   'userId': '57a0aa1071e394dd32ffe137',
   'description': '',
   'source': '',
   'authors': '',
   'application': ['rw'],
   'verified': False,
   'default': False,
   'protected': False,
   'defaultEditableWidget': False,
   'published': True,
   'freeze': False,
   'env': 'production',
   'queryUrl': '',
   'widgetConfig': {'lng': 4.504394531250001,
    'lat': -28.497660832963472,
    'zoom': 4,
    'type': 'map',
    'paramsConfig': {'embed': {'src': ''},
     'band': None,
     'areaIntersection': None,
     'filters': [],
     'chartType': None,
     'aggregateFunction': None,
     'orderBy': None,
     'size': None,
     'color': None,
     'category': None,
     'value': None,
     'limit': 500,
     'visualizationType': 'map',
     'layer_id': '8339b4a5-b99f-4d6

# River Water Quality

## Sediment Pressure TBD

In [ ]:
DatasetId = ''
layers = ['','','']
widgetBody['name'] = "Reduction in Life Expectancy Map"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(datasetId, widgetBody, 'staging')

Water quality Pressuere TBD

In [ ]:
DatasetId = ''
layers = ['','','']
widgetBody['name'] = "Reduction in Life Expectancy Map"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(datasetId, widgetBody, 'staging')

Ecosystems

In [ ]:
DatasetId = ''
layers = ['','','']
widgetBody['name'] = "Reduction in Life Expectancy Map"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(datasetId, widgetBody, 'staging')

In [ ]:
DatasetId = ''
layers = ['','','']
widgetBody['name'] = "Reduction in Life Expectancy Map"
widgetBody['widgetConfig']['paramsConfig']['layers'] = layers
widgetBody['widgetConfig']['paramsConfig']['layer'] = layers[0]
widgetBody['widgetConfig']['paramsConfig']['layer_id'] = layers[0]
createWidget(datasetId, widgetBody, 'staging')